# Scanse Sweep Tests

Simple test of Scanse Sweep:
* create scanner and scan getter threads
* gather 10 scans
* extract scans into `numpy` arrays
* plot a polar plot using `maptlotlib`

In [ ]:
import numpy as np
from sweeppy import Sweep
from matplotlib import pyplot as plt
import time
import queue
import threading
import sys


In [ ]:
def parse_scan(scan):
    angle = []
    distance = []
    strength=[]
    
    for sample in scan.samples:
        angle.append(sample.angle)
        distance.append(sample.distance)
        strength.append(sample.signal_strength )
        
    return np.array(angle), np.array(distance), np.array(strength)

In [ ]:
class Scanner(threading.Thread):
    """
    A Scanner thread gets scans fom the sensor and puts them into a FIFO queue
    """
    def __init__(self, dev, queue, stop_criterion):
        super().__init__()
        self.dev = dev
        self.queue = queue
        self.counter = 0
        self.stop_criterion = stop_criterion
        
    # Iterate over an infinite scan generator. Stop when asked to do so
    def run(self):
        with Sweep(self.dev) as sweep:
            sweep.start_scanning()

            for scan in sweep.get_scans():
                if self.counter == self.stop_criterion:
                    self.queue.put_nowait(None)
                    print("gathered {} scans".format(self.counter))
                    break
                else:
                    self.queue.put_nowait(scan)
                    self.counter += 1

            sweep.stop_scanning()


class ScanGetter(threading.Thread):
    """
    ScanGetter extracts scans from the referenced queue.
    """
    def __init__(self, queue):
        super().__init__()
        self.queue = queue
        self.scans = []

    # Iterate over the queue's scans blocking if the queue is
    # empty. If the value in queue is None leave thread.

    def run(self):
        while True:
            scan = self.queue.get()

            if not scan:
                break
        
            self.scans.append(scan)
            
            self.queue.task_done()

In [ ]:
dev = 'COM3'

fifo = queue.Queue()

scanner = Scanner(dev, fifo, 10)
counter = SampleCounter(fifo)

scanner.start()
counter.start()


In [ ]:
angle = []
distance = []
for scan in counter.scans:
    a,d, s = parse_scan(scan)
    angle.append(a)
    distance.append(d)

angle = np.hstack(angle)
distance = np.hstack(distance)

In [ ]:
plt.figure(figsize=(8,8))
plt.polar(np.radians(angle/1e3),distance, marker='.', linestyle='')